In [ ]:
%pip install --upgrade plotly numpy nbformat kaleido pandas 

In [ ]:
import plotly.express as px
import pandas as pd

In [ ]:
colors_map = {
    "IEEE": "#a80ca8",
    "Verrou": "#ff7f0e",
    "PRISM": "#1f77b4",
    "CESTAC": "#1fb438",
    "FM SR": "#26b7e4",
    "MCA RR": "#f10202",
}

In [ ]:
# Data extracted from the table
use_cases = ["MNIST", "FastSurfer", "WavLM"]

data = pd.read_csv("perf-DL-stats.csv")

color_discrete_map = {
    "IEEE": colors_map["IEEE"],
    "Verrou": colors_map["Verrou"],
    "PRISM": colors_map["PRISM"],
}

fig = px.bar(
    data,
    x="Benchmark",
    y="Mean",
    error_y="Std",
    color="Tool",
    barmode="group",
    pattern_shape="Variant",
    log_y=True,
    color_discrete_map=color_discrete_map,
    pattern_shape_sequence=["", "/", "", "/", ""],
)
# Combine 'Verrou CESTAC' and 'Verrou SR' into a single legend group called 'Verrou'
for trace in fig.data:
    tool = trace.hovertemplate.split("Tool=")[1].split("<br>")[0]
    variant = trace.hovertemplate.split("Variant=")[1].split("<br>")[0].strip()
    if trace.name.startswith("Verrou"):
        trace.legendgroup = "Verrou"
        trace.showlegend = True
        trace.name = f"{tool} {variant}"
    elif trace.name.startswith("PRISM"):
        trace.legendgroup = "PRISM"
        trace.showlegend = True
        trace.name = f"{tool} {variant}"
    else:
        trace.legendgroup = "IEEE"
        trace.showlegend = True
        trace.name = f"{tool}"

fig.update_yaxes(exponentformat="power")
fig.update_layout(
    xaxis_title="",
    yaxis_title="Mean Runtime (sec)",
    height=600,
    width=1200,
    legend=dict(
        title="Method",
        title_side="left",
        orientation="h",
        yanchor="bottom",
        y=1.02,
        valign="middle",
        xanchor="center",
        x=0.5,
    ),
    margin=dict(t=50, b=0, l=50, r=0),
    font=dict(size=20, family="Helvetica"),
)
# Reorder legend: PRISM UD before PRISM SR
# Find PRISM traces and swap their order in fig.data
prism_ud_idx = None
prism_sr_idx = None
for i, trace in enumerate(fig.data):
    if trace.name == "PRISM UD":
        prism_ud_idx = i
    elif trace.name == "PRISM SR":
        prism_sr_idx = i

if (
    prism_ud_idx is not None
    and prism_sr_idx is not None
    and prism_ud_idx > prism_sr_idx
):
    # Swap traces so PRISM UD comes before PRISM SR
    traces = list(fig.data)
    traces[prism_sr_idx], traces[prism_ud_idx] = (
        traces[prism_ud_idx],
        traces[prism_sr_idx],
    )
    fig.data = tuple(traces)


fig.show()